# JB2 - CIMR, RSIR and L1C

The goal of this notebook is to demonstrate how to perform an RSIR regridding on CIMR data to an EASE2 global grid. We will first outline the general steps for an L-band regrid. We will then compare this with a C and Ka band regrid. Finally, we will inspect the visual difference of the effect of changing the RSIR iteration parameter. 

# Step 1 - Import relevant modules

In [1]:
# Import Relevant Modules
from cimr_rgb.config_file import ConfigFile
from cimr_rgb.data_ingestion import DataIngestion
from cimr_rgb.regridder import ReGridder
from cimr_rgb.grid_generator import GRIDS
from cimr_rgb.product_generator import ProductGenerator

import copy
from numpy import full, nan, nanmean, isnan, isinf, polyfit

# You will need to configure matplotlib for your machine, you can probably remove the two lines below. 
import matplotlib # Remove this is plotting issues. 
tkagg = matplotlib.use('TkAgg') # Remove this if plotting issues. 
import matplotlib.pyplot as plt
plt.ion()

# Step 2 - Add your paths to the configuration file, load the configuration file and validate the configuration file 

In [2]:
# Define path for configuration file of the first regrid.
config_1_path = '/home/beywood/Desktop/MS3/CIMR-RGB/notebooks/JB2 - CIMR RSIR L1C/JB2.xml'

# Validate your config file
config_object = ConfigFile(config_1_path)

# You can inspect the parameters in the config file. There are additional parameters relevant to your particular configuration
# that are added as part of the validation. 
for attr, value in config_object.__dict__.items():
    print(f"{attr}: {value}")

/home/beywood/ST/CIMR_RGB/CIMR-RGB/dpr/L1B/CIMR/SCEPS_l1b_sceps_geo_central_america_scene_1_unfiltered_tot_minimal_nom_nedt_apc_tot_v2p1.nc
output_path: /home/beywood/Desktop/MS3/CIMR-RGB/notebooks/JB2 - CIMR RSIR L1C/insert path to output directory
timestamp: 2024-12-11_13-44-34
timestamp_fmt: %Y-%m-%d_%H-%M-%S
logpar_config: {'version': 1, 'disable_existing_loggers': False, 'loggers': {'rgb-logger': {'level': 'INFO', 'handlers': ['stdout', 'file'], 'propagate': False}}, 'handlers': {'stdout': {'class': 'logging.StreamHandler', 'formatter': 'simple', 'stream': 'ext://sys.stdout', 'level': 'INFO'}, 'file': {'class': 'logging.FileHandler', 'level': 'INFO', 'formatter': 'balanced', 'filename': PosixPath('/home/beywood/Desktop/MS3/CIMR-RGB/notebooks/JB2 - CIMR RSIR L1C/insert path to output directory/logs/JB2.log'), 'mode': 'w'}}, 'formatters': {'simple': {'format': '[%(name)s - %(levelname)s]: %(message)s'}, 'simple2': {'format': '[%(name)s - %(levelname)s | %(module)s - L%(lineno)d]: %(

# Step 2 - L-band

We will first outline the general steps for a regrid. When you run the code block below, you will notice that it will take a long time to process. This is unforatunate fact of the RSIR algorithm. Run the cell below and have a look at the percentage progress at the bottom of the terminal. As you probably don't want to wait a few hours, we have functionality that allows you to only regrid a part of the grid for quick inspection. 

After running the block below, if you don't want to wait for the entire granule to be processed, press the stop button at the top of the page and continue to the next block. (it may also be useful to comment this code block before continuing) 

In [3]:
# We validated the config object in the block above, we will now use this config object to ingest data and perform a regrid

# Ingest Data 
# l1b_data = DataIngestion(config_object).ingest_data()

# Regrid data 
# l1c_l_band = ReGridder(config_object).regrid_data()

# reduced_grid_inds

We now want to only regrid a part of the output grid so we can do some quick inspection. In order to determine where on the EASE grid the data lands, and therefore select appropriate reduced_grid_inds parameters, we do a quick IDS regrid and inspect the result. 

The run should take around 2 minnutes, if you want a faster run, just make the reduced_grid_inds smaller. 

In [4]:
# We edit the config object to perform a IDS regrid. 

ids_config = copy.deepcopy(config_object)
ids_config.regridding_algorithm = 'IDS'


# Perform the IDS regrid 
l1b_data = DataIngestion(config_object).ingest_data()
l1c_ids = ReGridder(ids_config).regrid_data(l1b_data)

# Inspect the result on the EASE grid. 
variable = l1c_ids['L']['bt_h']
cell_row = l1c_ids['L']['cell_row']
cell_col = l1c_ids['L']['cell_col']

# Create grid
grid_shape = GRIDS[ids_config.grid_definition]['n_rows'], GRIDS[ids_config.grid_definition]['n_cols']
ease_grid = full(grid_shape, nan)
# Construct Grid 
for i in range(len(cell_row)):
    ease_grid[cell_row[i], cell_col[i]] = variable[i]

fig, axs = plt.subplots()
axs.imshow(ease_grid)
plt.show()


[rgb-logger - INFO]: `L` Band: Calculating max altitude for `ap_radius`
[rgb-logger - INFO]: `combine_cimr_feeds`
[rgb-logger - INFO]: ---------------------
[rgb-logger - INFO]: `combine_cimr_feeds` -- Started Execution
[rgb-logger - INFO]: `combine_cimr_feeds` -- Executed in: 0.01s
[rgb-logger - INFO]: `combine_cimr_feeds` -- CPU User Time (Change): 0.01s
[rgb-logger - INFO]: `combine_cimr_feeds` -- CPU System Time: 0.00s
[rgb-logger - INFO]: `combine_cimr_feeds` -- CPU Total Time: 0.01s
[rgb-logger - INFO]: `combine_cimr_feeds` -- Process-Specific CPU Usage (Before): 0.00%
[rgb-logger - INFO]: `combine_cimr_feeds` -- Process-Specific CPU Usage (After): 115.20%
[rgb-logger - INFO]: `combine_cimr_feeds` -- Memory Usage Change: 5.000000 MB
[rgb-logger - INFO]: ---------------------
[rgb-logger - INFO]: `remove_out_of_bounds`
[rgb-logger - INFO]: ---------------------
[rgb-logger - INFO]: `remove_out_of_bounds` -- Started Execution
[rgb-logger - INFO]: `GridGenerator`
[rgb-logger - INFO]

We can see from the image where the data lies on the grid and can now use the axes to select an appropriate smaller range to inspect data for the RSIR algorithm. We zoom into the region around around the border in the South of Mexico and choose rows from 540 to 600 and cols from 890 to 940. Now we need to update our config object and run again. 

In [5]:
rsir_reduced_object = copy.deepcopy(ids_config)
rsir_reduced_object.regridding_algorithm = 'RSIR'
rsir_reduced_object.reduced_grid_inds = [560, 580, 900, 920]

l1c_rsir_reduced = ReGridder(rsir_reduced_object).regrid_data(l1b_data)

# Inspect the new data 
# Inspect the result on the EASE grid. 
variable = l1c_rsir_reduced['L']['bt_h']
cell_row = l1c_rsir_reduced['L']['cell_row']
cell_col = l1c_rsir_reduced['L']['cell_col']

# Create grid
grid_shape = GRIDS[ids_config.grid_definition]['n_rows'], GRIDS[ids_config.grid_definition]['n_cols']
ease_grid = full(grid_shape, nan)
# Construct Grid 
for i in range(len(cell_row)):
    ease_grid[cell_row[i], cell_col[i]] = variable[i]

fig, axs = plt.subplots()
axs.imshow(ease_grid[560:580, 900:920])
plt.show()

[rgb-logger - INFO]: `get_grid`
[rgb-logger - INFO]: ---------------------
[rgb-logger - INFO]: `get_grid` -- Started Execution
[rgb-logger - INFO]: `generate_grid_xy`
[rgb-logger - INFO]: ---------------------
[rgb-logger - INFO]: `generate_grid_xy` -- Started Execution
[rgb-logger - INFO]: `generate_grid_xy_ease2`
[rgb-logger - INFO]: ---------------------
[rgb-logger - INFO]: `generate_grid_xy_ease2` -- Started Execution
[rgb-logger - INFO]: `generate_grid_xy_ease2` -- Executed in: 0.01s
[rgb-logger - INFO]: `generate_grid_xy_ease2` -- CPU User Time (Change): 0.01s
[rgb-logger - INFO]: `generate_grid_xy_ease2` -- CPU System Time: 0.00s
[rgb-logger - INFO]: `generate_grid_xy_ease2` -- CPU Total Time: 0.01s
[rgb-logger - INFO]: `generate_grid_xy_ease2` -- Process-Specific CPU Usage (Before): 0.00%
[rgb-logger - INFO]: `generate_grid_xy_ease2` -- Process-Specific CPU Usage (After): 176.70%
[rgb-logger - INFO]: `generate_grid_xy_ease2` -- Memory Usage Change: 0.000000 MB
[rgb-logger - I

  0%|                                                   | 0/404 [00:00<?, ?it/s]

RSIR_ITERATION = 15
[rgb-logger - WARNING]: RuntimeWarning: invalid value encountered in divide (File: /home/beywood/Desktop/MS3/CIMR-RGB/src/cimr_rgb/rsir.py, Line: 197)


  1%|▎                                          | 3/404 [00:00<00:14, 28.14it/s]

RSIR_ITERATION = 15
RSIR_ITERATION = 15
RSIR_ITERATION = 15
RSIR_ITERATION = 15


...

 99%|████████████████████████████████████████▍| 399/404 [00:16<00:00, 26.36it/s]

RSIR_ITERATION = 15
RSIR_ITERATION = 15
RSIR_ITERATION = 15
RSIR_ITERATION = 15
RSIR_ITERATION = 15
RSIR_ITERATION = 15


100%|█████████████████████████████████████████| 404/404 [00:16<00:00, 24.86it/s]

RSIR_ITERATION = 15
RSIR_ITERATION = 15
RSIR_ITERATION = 15
RSIR_ITERATION = 15
RSIR_ITERATION = 15
[rgb-logger - INFO]: `interp_variable_dict` -- Executed in: 16.25s
[rgb-logger - INFO]: `interp_variable_dict` -- CPU User Time (Change): 7.37s
[rgb-logger - INFO]: `interp_variable_dict` -- CPU System Time: 8.94s


[rgb-logger - INFO]: `interp_variable_dict` -- CPU Total Time: 16.31s
[rgb-logger - INFO]: `interp_variable_dict` -- Process-Specific CPU Usage (Before): 0.00%
[rgb-logger - INFO]: `interp_variable_dict` -- Process-Specific CPU Usage (After): 100.30%
[rgb-logger - INFO]: `interp_variable_dict` -- Memory Usage Change: 1.339844 MB
[rgb-logger - INFO]: ---------------------
[rgb-logger - INFO]: `create_output_grid_inds`
[rgb-logger - INFO]: ---------------------
[rgb-logger - INFO]: `create_output_grid_inds` -- Started Execution
[rgb-logger - INFO]: `create_output_grid_inds` -- Executed in: 0.00s
[rgb-logger - INFO]: `create_output_grid_inds` -- CPU User Time (Change): 0.00s
[rgb-logger - INFO]: `create_output_grid_inds` -- CPU System Time: 0.00s
[rgb-logger - INFO]: `create_output_grid_inds` -- CPU Total Time: 0.00s
[rgb-logger - INFO]: `create_output_grid_inds` -- Process-Specific CPU Usage (Before): 0.00%
[rgb-logger - INFO]: `create_output_grid_inds` -- Process-Specific CPU Usage (Aft

# Comparison of L band with C band and Ka band 

Note, the regridding of a few bands will take longer!

In [6]:
multiband_config = copy.deepcopy(rsir_reduced_object)
multiband_config.target_band = ['L', 'C', 'KA']

l1b_multi = DataIngestion(multiband_config).ingest_data()
l1c_multi = ReGridder(multiband_config).regrid_data(l1b_multi)


[rgb-logger - INFO]: `KA` Band: Calculating max altitude for `ap_radius`
[rgb-logger - INFO]: `combine_cimr_feeds`
[rgb-logger - INFO]: ---------------------
[rgb-logger - INFO]: `combine_cimr_feeds` -- Started Execution
[rgb-logger - INFO]: `combine_cimr_feeds` -- Executed in: 0.31s
[rgb-logger - INFO]: `combine_cimr_feeds` -- CPU User Time (Change): 0.13s
[rgb-logger - INFO]: `combine_cimr_feeds` -- CPU System Time: 0.18s
[rgb-logger - INFO]: `combine_cimr_feeds` -- CPU Total Time: 0.31s
[rgb-logger - INFO]: `combine_cimr_feeds` -- Process-Specific CPU Usage (Before): 0.00%
[rgb-logger - INFO]: `combine_cimr_feeds` -- Process-Specific CPU Usage (After): 99.40%
[rgb-logger - INFO]: `combine_cimr_feeds` -- Memory Usage Change: 723.136719 MB
[rgb-logger - INFO]: ---------------------
[rgb-logger - INFO]: `remove_out_of_bounds`
[rgb-logger - INFO]: ---------------------
[rgb-logger - INFO]: `remove_out_of_bounds` -- Started Execution
[rgb-logger - INFO]: `GridGenerator`
[rgb-logger - INF

  1%|▎                                          | 3/441 [00:00<00:17, 25.07it/s]

RSIR_ITERATION = 15
RSIR_ITERATION = 15
RSIR_ITERATION = 15
RSIR_ITERATION = 15
RSIR_ITERATION = 15
RSIR_ITERATION = 15


  2%|█                                         | 11/441 [00:00<00:14, 28.91it/s]

RSIR_ITERATION = 15
RSIR_ITERATION = 15
RSIR_ITERATION = 15
RSIR_ITERATION = 15
RSIR_ITERATION = 15
RSIR_ITERATION = 15


...

RSIR_ITERATION = 15
RSIR_ITERATION = 15
RSIR_ITERATION = 15
RSIR_ITERATION = 15
RSIR_ITERATION = 15
RSIR_ITERATION = 15
RSIR_ITERATION = 15
RSIR_ITERATION = 15
RSIR_ITERATION = 15


 99%|████████████████████████████████████████▍| 399/404 [00:10<00:00, 47.05it/s]

RSIR_ITERATION = 15
RSIR_ITERATION = 15
RSIR_ITERATION = 15
RSIR_ITERATION = 15
RSIR_ITERATION = 15
RSIR_ITERATION = 15
RSIR_ITERATION = 15
RSIR_ITERATION = 15
RSIR_ITERATION = 15
RSIR_ITERATION = 15


100%|█████████████████████████████████████████| 404/404 [00:10<00:00, 38.69it/s]

RSIR_ITERATION = 15
RSIR_ITERATION = 15
RSIR_ITERATION = 15
RSIR_ITERATION = 15
[rgb-logger - INFO]: `interp_variable_dict` -- Executed in: 10.45s
[rgb-logger - INFO]: `interp_variable_dict` -- CPU User Time (Change): 10.36s
[rgb-logger - INFO]: `interp_variable_dict` -- CPU System Time: 0.05s
[rgb-logger - INFO]: `interp_variable_dict` -- CPU Total Time: 10.41s
[rgb-logger - INFO]: `interp_variable_dict` -- Process-Specific CPU Usage (Before): 0.00%
[rgb-logger - INFO]: `interp_variable_dict` -- Process-Specific CPU Usage (After): 99.70%
[rgb-logger - INFO]: `interp_variable_dict` -- Memory Usage Change: 0.000000 MB
[rgb-logger - INFO]: ---------------------
[rgb-logger - INFO]: `create_output_grid_inds`
[rgb-logger - INFO]: ---------------------
[rgb-logger - INFO]: `create_output_grid_inds` -- Started Execution
[rgb-logger - INFO]: `create_output_grid_inds` -- Executed in: 0.00s
[rgb-logger - INFO]: `create_output_grid_inds` -- CPU User Time (Change): 0.00s
[rgb-logger - INFO]: `cre

In [7]:
# plotting 

grid_shape = GRIDS[multiband_config.grid_definition]['n_rows'], GRIDS[multiband_config.grid_definition]['n_cols']
L_grid, C_grid, KA_grid = full(grid_shape, nan), full(grid_shape, nan), full(grid_shape, nan)

L_band_data = l1c_multi['L']
bt_h_l = L_band_data['bt_h']
cell_col_l = L_band_data['cell_col']
cell_row_l = L_band_data['cell_row']

for i in range(len(cell_col_l)):
    L_grid[cell_row_l[i], cell_col_l[i]] = bt_h_l[i]

C_band_data = l1c_multi['C']
bt_h_c = C_band_data['bt_h']
cell_col_c = C_band_data['cell_col']
cell_row_c = C_band_data['cell_row']

for i in range(len(cell_col_c)):
    C_grid[cell_row_c[i], cell_col_c[i]] = bt_h_c[i]

KA_band_data = l1c_multi['KA']
bt_h_ka = KA_band_data['bt_h']
cell_col_ka = KA_band_data['cell_col']
cell_row_ka = KA_band_data['cell_row']

for i in range(len(cell_col_ka)):
    KA_grid[cell_row_ka[i], cell_col_ka[i]] = bt_h_ka[i]

fig, axs = plt.subplots(1, 3)
im1 = axs[0].imshow(L_grid[560:580, 900:920], cmap = 'viridis')
axs[0].set_title('L_BAND')
im2 = axs[1].imshow(C_grid[560:580, 900:920], cmap = 'viridis')
axs[1].set_title('C_BAND')
im3 = axs[2].imshow(KA_grid[560:580, 900:920], cmap = 'viridis')
axs[2].set_title('KA_BAND')
colorbar = fig.colorbar(im1, ax=axs[0])
colorbar = fig.colorbar(im2, ax=axs[1])
colorbar = fig.colorbar(im3, ax=axs[2])
colorbar.set_label('BT [K]', fontsize=12)
plt.show()

# Inspect effect of RSIR Iteration. 

Now we will do three regrids of K band, but inspect a changing iteration parameter. 

In [8]:
iteration_parameter = [1, 5, 15]

config_ka = copy.deepcopy(multiband_config)
config_ka.target_band = ['KA']

l1b_ka = DataIngestion(config_ka).ingest_data()

grid_shape = GRIDS[config_ka.grid_definition]['n_rows'], GRIDS[config_ka.grid_definition]['n_cols']


output_data = {}
for i in iteration_parameter:
    iteration_data = {}
    config = copy.deepcopy(config_ka)
    config.rsir_iteration = i
    l1c_ka = ReGridder(config).regrid_data(l1b_ka)
    bt_h = l1c_ka['KA']['bt_h']
    cell_row = l1c_ka['KA']['cell_row']
    cell_col = l1c_ka['KA']['cell_col']
    grid = full(grid_shape, nan)
    
    for sample in range(len(cell_row)):
        grid[cell_row[sample], cell_col[sample]] = bt_h[sample]

    output_data[str(i)] = grid

    

[rgb-logger - INFO]: `combine_cimr_feeds`
[rgb-logger - INFO]: ---------------------
[rgb-logger - INFO]: `combine_cimr_feeds` -- Started Execution
[rgb-logger - INFO]: `combine_cimr_feeds` -- Executed in: 0.34s
[rgb-logger - INFO]: `combine_cimr_feeds` -- CPU User Time (Change): 0.15s
[rgb-logger - INFO]: `combine_cimr_feeds` -- CPU System Time: 0.18s
[rgb-logger - INFO]: `combine_cimr_feeds` -- CPU Total Time: 0.33s
[rgb-logger - INFO]: `combine_cimr_feeds` -- Process-Specific CPU Usage (Before): 0.00%
[rgb-logger - INFO]: `combine_cimr_feeds` -- Process-Specific CPU Usage (After): 98.30%
[rgb-logger - INFO]: `combine_cimr_feeds` -- Memory Usage Change: 745.515625 MB
[rgb-logger - INFO]: ---------------------
[rgb-logger - INFO]: `remove_out_of_bounds`
[rgb-logger - INFO]: ---------------------
[rgb-logger - INFO]: `remove_out_of_bounds` -- Started Execution
[rgb-logger - INFO]: `GridGenerator`
[rgb-logger - INFO]: ---------------------
[rgb-logger - INFO]: `GridGenerator` -- Started

  1%|▎                                          | 3/441 [00:00<00:16, 27.05it/s]

RSIR_ITERATION = 1
RSIR_ITERATION = 1
RSIR_ITERATION = 1
RSIR_ITERATION = 1
RSIR_ITERATION = 1
RSIR_ITERATION = 1


  2%|█                                         | 11/441 [00:00<00:13, 31.71it/s]

RSIR_ITERATION = 1
RSIR_ITERATION = 1
RSIR_ITERATION = 1
RSIR_ITERATION = 1
RSIR_ITERATION = 1
RSIR_ITERATION = 1
RSIR_ITERATION = 1
RSIR_ITERATION = 1


...

RSIR_ITERATION = 15
RSIR_ITERATION = 15
RSIR_ITERATION = 15
RSIR_ITERATION = 15
RSIR_ITERATION = 15


 99%|████████████████████████████████████████▌| 436/441 [00:28<00:00, 21.55it/s]

RSIR_ITERATION = 15
RSIR_ITERATION = 15
RSIR_ITERATION = 15
RSIR_ITERATION = 15
RSIR_ITERATION = 15


100%|█████████████████████████████████████████| 441/441 [00:29<00:00, 15.15it/s]

RSIR_ITERATION = 15
RSIR_ITERATION = 15
RSIR_ITERATION = 15
RSIR_ITERATION = 15
[rgb-logger - INFO]: `interp_variable_dict` -- Executed in: 29.12s
[rgb-logger - INFO]: `interp_variable_dict` -- CPU User Time (Change): 16.59s
[rgb-logger - INFO]: `interp_variable_dict` -- CPU System Time: 12.40s
[rgb-logger - INFO]: `interp_variable_dict` -- CPU Total Time: 28.99s
[rgb-logger - INFO]: `interp_variable_dict` -- Process-Specific CPU Usage (Before): 0.00%
[rgb-logger - INFO]: `interp_variable_dict` -- Process-Specific CPU Usage (After): 99.60%
[rgb-logger - INFO]: `interp_variable_dict` -- Memory Usage Change: 1.367188 MB
[rgb-logger - INFO]: ---------------------
[rgb-logger - INFO]: `create_output_grid_inds`
[rgb-logger - INFO]: ---------------------
[rgb-logger - INFO]: `create_output_grid_inds` -- Started Execution
[rgb-logger - INFO]: `create_output_grid_inds` -- Executed in: 0.00s
[rgb-logger - INFO]: `create_output_grid_inds` -- CPU User Time (Change): 0.00s
[rgb-logger - INFO]: `cr

In [9]:
# Plotting 
fig, axs = plt.subplots(1, 3)
im1 = axs[0].imshow(output_data['1'][560:580, 900:920], cmap = 'viridis')
axs[0].set_title('iteration = 1')
im2 = axs[1].imshow(output_data['5'][560:580, 900:920], cmap = 'viridis')
axs[1].set_title('iteration = 5')
im3 = axs[2].imshow(output_data['15'][560:580, 900:920], cmap = 'viridis')
axs[2].set_title('iteration = 15')
colorbar = fig.colorbar(im1, ax=axs[0])
colorbar = fig.colorbar(im2, ax=axs[1])
colorbar = fig.colorbar(im3, ax=axs[2])
colorbar.set_label('BT [K]', fontsize=12)
plt.show()

{'1': array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]], shape=(1624, 3856)), '5': array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]], shape=(1624, 3856)), '15': array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]], shape=(1624, 3856))}
